# Import Libraries

In [1]:
import pandas as pd
import json
import numpy as np

# Ground truths

In [2]:
# Read jsonl file
solution = pd.read_json('../data_task1/MMDocIR_gt_by_id.jsonl', lines=True)

In [3]:
solution

,question_id,question,answer,doc_name,domain,num_of_pages,page_id,layout_mapping
0,10000,"In figure 1, which relation arrows do not poin...","['movement', 'birth', 'place of birth', 'member']",2310.05634v2,Academic paper,24,[0],"[{'page': 0, 'page_size': [595.2760009765625, ..."
1,10001,"In figure 5, what is the color of the line tha...",red,2310.05634v2,Academic paper,24,[7],"[{'page': 7, 'page_size': [595.2760009765625, ..."
2,10002,"How many tables include ""F1"" as a metric?",3,2310.05634v2,Academic paper,24,"[6, 8]","[{'page': 6, 'page_size': [595.2760009765625, ..."
3,10003,"From the paper, which temperature gives ChatGP...",0.1,2310.05634v2,Academic paper,24,[6],"[{'page': 6, 'page_size': [595.2760009765625, ..."
4,10004,"For dataset construction, which step takes the...",Evolutionary Question Generation,2310.05634v2,Academic paper,24,"[11, 12]","[{'page': 11, 'page_size': [595.2760009765625,..."
...,...,...,...,...,...,...,...,...
1653,11653,How many black arrows are part of the DISCIPLI...,2,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,41,[0],"[{'page': 0, 'page_size': [768.0, 594.0], 'bbo..."
1654,11654,How many different twitter accounts are shown ...,2,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,41,"[1, 2, 37]","[{'page': 1, 'page_size': [768.0, 594.0], 'bbo..."
1655,11655,How many different Chinese characters are show...,3,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,41,[38],"[{'page': 38, 'page_size': [768.0, 594.0], 'bb..."
1656,11656,"How many times does ""Barclays"" appear on page 4?",5,disciplined-agile-business-analysis-1602180127...,Tutorial/Workshop,41,[3],"[{'page': 3, 'page_size': [768.0, 594.0], 'bbo..."


In [4]:
solution = solution[["question_id", "page_id", "domain"]]
solution = solution.rename(columns = {"page_id" : "passage_id"})

In [5]:
solution

,question_id,passage_id,domain
0,10000,[0],Academic paper
1,10001,[7],Academic paper
2,10002,"[6, 8]",Academic paper
3,10003,[6],Academic paper
4,10004,"[11, 12]",Academic paper
...,...,...,...
1653,11653,[0],Tutorial/Workshop
1654,11654,"[1, 2, 37]",Tutorial/Workshop
1655,11655,[38],Tutorial/Workshop
1656,11656,[3],Tutorial/Workshop


In [6]:
type(solution["passage_id"][2])

list

In [7]:
solution[solution['question_id'].isin([10900, 10901, 10902])].index

Index([900, 901, 902], dtype='int64')

In [8]:
# Drop rows where question_id is 10900, 10901, 10902 - Since this document has only 3 pages in all
solution = solution.drop(index=solution[solution['question_id'].isin([10900, 10901, 10902])].index)
solution.reset_index(drop=True, inplace=True)

# MMDocIR Submission

In [9]:
submission = pd.read_csv("../submissions/submission_vidore_colqwen2-v1.0.csv")

In [10]:
# Just evaluate Task 1 - MMDocIR
submission = submission[6415:]
submission = submission.drop(index=submission[submission['question_id'].isin([10900, 10901, 10902])].index)
submission.reset_index(drop=True, inplace=True)

In [11]:
submission

,question_id,passage_id
0,10000,"[""11"", ""1"", ""3"", ""12"", ""4""]"
1,10001,"[""7"", ""4"", ""16"", ""20"", ""21""]"
2,10002,"[""6"", ""7"", ""8"", ""4"", ""5""]"
3,10003,"[""6"", ""5"", ""7"", ""8"", ""13""]"
4,10004,"[""2"", ""11"", ""1"", ""9"", ""13""]"
...,...,...
1650,11653,"[""38"", ""0"", ""20"", ""8"", ""17""]"
1651,11654,"[""37"", ""1"", ""2"", ""14"", ""30""]"
1652,11655,"[""38"", ""16"", ""14"", ""29"", ""4""]"
1653,11656,"[""3"", ""2"", ""0"", ""34"", ""33""]"


In [12]:
type(submission["passage_id"][45])

str

In [13]:
submission["passage_id"]

0         ["11", "1", "3", "12", "4"]
1        ["7", "4", "16", "20", "21"]
2           ["6", "7", "8", "4", "5"]
3          ["6", "5", "7", "8", "13"]
4         ["2", "11", "1", "9", "13"]
                    ...              
1650     ["38", "0", "20", "8", "17"]
1651     ["37", "1", "2", "14", "30"]
1652    ["38", "16", "14", "29", "4"]
1653      ["3", "2", "0", "34", "33"]
1654    ["9", "10", "23", "12", "13"]
Name: passage_id, Length: 1655, dtype: object

In [14]:
# Convert the 'passage_id' columns from stringified lists to actual lists
submission["passage_id"] = submission["passage_id"].apply(json.loads)

In [15]:
submission["passage_id"]

0         [11, 1, 3, 12, 4]
1        [7, 4, 16, 20, 21]
2           [6, 7, 8, 4, 5]
3          [6, 5, 7, 8, 13]
4         [2, 11, 1, 9, 13]
               ...         
1650     [38, 0, 20, 8, 17]
1651     [37, 1, 2, 14, 30]
1652    [38, 16, 14, 29, 4]
1653      [3, 2, 0, 34, 33]
1654    [9, 10, 23, 12, 13]
Name: passage_id, Length: 1655, dtype: object

In [16]:
submission["passage_id"] = submission["passage_id"].apply(lambda x: [int(ele) for ele in x])

In [17]:
type(submission["passage_id"][3])

list

In [18]:
submission["domain"] = solution["domain"]

In [19]:
submission

,question_id,passage_id,domain
0,10000,"[11, 1, 3, 12, 4]",Academic paper
1,10001,"[7, 4, 16, 20, 21]",Academic paper
2,10002,"[6, 7, 8, 4, 5]",Academic paper
3,10003,"[6, 5, 7, 8, 13]",Academic paper
4,10004,"[2, 11, 1, 9, 13]",Academic paper
...,...,...,...
1650,11653,"[38, 0, 20, 8, 17]",Tutorial/Workshop
1651,11654,"[37, 1, 2, 14, 30]",Tutorial/Workshop
1652,11655,"[38, 16, 14, 29, 4]",Tutorial/Workshop
1653,11656,"[3, 2, 0, 34, 33]",Tutorial/Workshop


In [20]:
solution

,question_id,passage_id,domain
0,10000,[0],Academic paper
1,10001,[7],Academic paper
2,10002,"[6, 8]",Academic paper
3,10003,[6],Academic paper
4,10004,"[11, 12]",Academic paper
...,...,...,...
1650,11653,[0],Tutorial/Workshop
1651,11654,"[1, 2, 37]",Tutorial/Workshop
1652,11655,[38],Tutorial/Workshop
1653,11656,[3],Tutorial/Workshop


# Average Recall@k

In [21]:
# # Define the K values to evaluate Recall@K
# ks = [1, 3, 5]

# evaluation = solution.copy()

# for k in ks:
#     evaluation[f"recall@{k}"] = 0.0

# # Initialize a dictionary to store recall scores for each K
# recall_scores = {f"recall@{k}": 0 for k in ks}

# for row_iloc,(true_passages, predicted_passages) in enumerate(zip(solution["passage_id"], submission["passage_id"])):
#     true_passages = set(true_passages)

#     if len(true_passages)>5:
#         print(true_passages, predicted_passages)
#         print("yes")
#     for k in ks:
#         top_k_predicted = set(predicted_passages[:k])
#         intersection = true_passages & top_k_predicted
#         recall = len(intersection) / len(true_passages)

#         evaluation.at[row_iloc, f"recall@{k}"] = recall

#         recall_scores[f"recall@{k}"] += recall

# num_questions_mmdocir = len(solution)
# avg_recall_mmdocir = sum(recall_scores[f"recall@{k}"] / num_questions_mmdocir for k in ks) / len(ks) * 100
# print(f"Recall@1: {recall_scores['recall@1'] / num_questions_mmdocir  * 100}")
# print(f"Recall@3: {recall_scores['recall@3'] / num_questions_mmdocir  * 100}")
# print(f"Recall@5: {recall_scores['recall@5'] / num_questions_mmdocir  * 100}")
# print(f"Avg Recall: {avg_recall_mmdocir}")

In [22]:
class retrieval_scores:

    def __init__(self, submission, solution, ks = [1, 3, 5]):
        self.submission = submission
        self.solution = solution
        self.ks = ks
        
    def average_recall(self):

        # Initialize a dictionary to store recall scores for each K
        recall_scores = {f"recall@{k}": [] for k in self.ks}
        avg_recall_scores = {f"recall@{k}": 0 for k in self.ks}

        for true_passages, predicted_passages in zip(self.solution["passage_id"], self.submission["passage_id"]):
            true_passages = set(true_passages)

            for k in self.ks:
                top_k_predicted = set(predicted_passages[:k])
                intersection = true_passages & top_k_predicted
                recall = len(intersection) / len(true_passages)

                recall_scores[f"recall@{k}"].append(recall)

        num_questions_per_domain = len(self.solution)
        
        for k in self.ks:
            avg_recall_scores[f"recall@{k}"] = 100 * (sum(recall_scores[f'recall@{k}'])/len(recall_scores[f'recall@{k}']))
            # print(f"Recall@{k}: {avg_recall_scores[f'recall@{k}']}")
            

        avg_recall_domain = {"Average" : sum(avg_recall_scores.values())/len(self.ks)}
        # print(f"Average of all: {avg_recall_domain['Average']}")

        return (avg_recall_domain, avg_recall_scores, num_questions_per_domain)
    

    def avg_precision(self, ground_truth_list, retrieved_list):

        precision_sum = 0
        hits = 0
        for ret_idx, retrieved_doc in enumerate(retrieved_list):
            # rel = 0
            
            if retrieved_doc in ground_truth_list:
                # rel = 1
                hits += 1

                precision_at_k = hits/(ret_idx + 1)

                precision_sum += precision_at_k

        total_relevant_docs = len(ground_truth_list)

        if total_relevant_docs==0:
            return 0.0
        
        AP = precision_sum / total_relevant_docs
        return AP

    def mAP(self):
        # Initialize a list to store average precision scores for each query
        AP_scores = []

        for true_passages, predicted_passages in zip(self.solution["passage_id"], self.submission["passage_id"]):

            AP_scores.append(self.avg_precision(true_passages, predicted_passages))

        return sum(AP_scores)/len(AP_scores)


In [23]:
# Overall Metric
fetch_metric = retrieval_scores(submission, solution)

print("*-"*25)
print(f"Domain : All included")

recall_metrics = fetch_metric.average_recall()
print(recall_metrics)
mAP_metric = fetch_metric.mAP()
print(f"mAP : {mAP_metric}")

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Domain : All included
({'Average': 76.014434373951}, {'recall@1': 61.459214501510615, 'recall@3': 80.8318227593152, 'recall@5': 85.75226586102718}, 1655)
mAP : 0.7358350117489099


In [24]:
recall_metrics

({'Average': 76.014434373951},
 {'recall@1': 61.459214501510615,
  'recall@3': 80.8318227593152,
  'recall@5': 85.75226586102718},
 1655)

In [25]:
# Domain-Wise Metrics

domains = ['Research report / Introduction', 'Administration/Industry file', 'Tutorial/Workshop', 'Academic paper', 
           'Brochure', 'Financial report', 'Guidebook', 'Government', 'Laws', 'News']
for domain in domains:
    subset_submission_df = submission[submission["domain"]== domain]
    subset_solution_df = solution[solution["domain"]== domain]

    fetch_metric = retrieval_scores(subset_submission_df, subset_solution_df)

    print("*-"*25)
    print(f"Domain : {domain}")

    recall_metrics = fetch_metric.average_recall()
    print(recall_metrics)
    mAP_metric = fetch_metric.mAP()
    print(f"mAP : {mAP_metric}")

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Domain : Research report / Introduction
({'Average': 74.42439862542955}, {'recall@1': 56.95017182130584, 'recall@3': 78.6855670103093, 'recall@5': 87.63745704467352}, 194)
mAP : 0.7452333906071017
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Domain : Administration/Industry file
({'Average': 74.28571428571429}, {'recall@1': 49.910714285714285, 'recall@3': 84.25595238095237, 'recall@5': 88.6904761904762}, 56)
mAP : 0.6979960317460318
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Domain : Tutorial/Workshop
({'Average': 73.91025641025641}, {'recall@1': 59.32692307692309, 'recall@3': 78.14102564102564, 'recall@5': 84.2628205128205}, 104)
mAP : 0.7656250000000001
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Domain : Academic paper
({'Average': 82.00228506141103}, {'recall@1': 65.10711225364182, 'recall@3': 87.97343616109684, 'recall@5': 92.92630676949443}, 389)
mAP : 0.7884233076263925
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-

# Ignore after this

In [26]:
# Define the K values to evaluate Recall@K
ks = [1, 3, 5, 10, 15]

print(f"Recall@10: {recall_scores['recall@10'] / num_questions_mmdocir  * 100}")
print(f"Recall@15: {recall_scores['recall@15'] / num_questions_mmdocir  * 100}")

NameError: name 'recall_scores' is not defined